In [1]:
# Import required libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer, pipeline
from peft import get_peft_model, LoraConfig
import evaluate
import numpy as np
from huggingface_hub import notebook_login

In [2]:
raw_datasets = load_dataset("conll2003")
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [3]:
# Look at the tokens of the first training example
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [4]:
# Look at the NER tags of the first training example
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [5]:
# Get the label names for the NER tags
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [6]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [7]:
# Load the tokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

e:\open_source\peft-folder\ner-examples\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Tokenize the first training example
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [9]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [10]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [12]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


In [15]:
metric = evaluate.load("seqeval")

In [16]:
# Create label mappings
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [17]:
# Load the pre-trained model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.config.num_labels

9

In [19]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [20]:
# Configure LoRA (Low-Rank Adaptation) for fine-tuning
peft_config = LoraConfig(target_modules = ["query", "key"])

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 108,021,513 || trainable%: 0.2730


In [21]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [22]:
notebook_login()

In [27]:
args = TrainingArguments(
    "bert-finetuned-ner-lora",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
)

In [28]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

  0%|          | 0/17560 [00:00<?, ?it/s]

{'loss': 1.8459, 'grad_norm': 0.37855836749076843, 'learning_rate': 1.9430523917995446e-05, 'epoch': 0.28}
{'loss': 1.6246, 'grad_norm': 0.39867204427719116, 'learning_rate': 1.886104783599089e-05, 'epoch': 0.57}
{'loss': 1.4456, 'grad_norm': 0.44621285796165466, 'learning_rate': 1.8291571753986334e-05, 'epoch': 0.85}


  0%|          | 0/407 [00:00<?, ?it/s]

{'eval_runtime': 5.1096, 'eval_samples_per_second': 636.059, 'eval_steps_per_second': 79.654, 'epoch': 1.0}
{'loss': 1.2745, 'grad_norm': 0.6124112010002136, 'learning_rate': 1.7722095671981778e-05, 'epoch': 1.14}
{'loss': 1.1606, 'grad_norm': 0.5666603446006775, 'learning_rate': 1.7152619589977222e-05, 'epoch': 1.42}
{'loss': 1.0874, 'grad_norm': 0.614331841468811, 'learning_rate': 1.6583143507972667e-05, 'epoch': 1.71}
{'loss': 1.0344, 'grad_norm': 0.8482342958450317, 'learning_rate': 1.601366742596811e-05, 'epoch': 1.99}


  0%|          | 0/407 [00:00<?, ?it/s]

...

{'eval_runtime': 5.0887, 'eval_samples_per_second': 638.671, 'eval_steps_per_second': 79.981, 'epoch': 9.0}
{'loss': 0.7176, 'grad_norm': 0.9316858053207397, 'learning_rate': 1.7767653758542143e-06, 'epoch': 9.11}
{'loss': 0.7109, 'grad_norm': 0.7026327848434448, 'learning_rate': 1.2072892938496584e-06, 'epoch': 9.4}
{'loss': 0.7088, 'grad_norm': 1.1022803783416748, 'learning_rate': 6.378132118451026e-07, 'epoch': 9.68}
{'loss': 0.7103, 'grad_norm': 0.578183650970459, 'learning_rate': 6.83371298405467e-08, 'epoch': 9.97}


  0%|          | 0/407 [00:00<?, ?it/s]

{'eval_runtime': 5.071, 'eval_samples_per_second': 640.904, 'eval_steps_per_second': 80.261, 'epoch': 10.0}
{'train_runtime': 641.6774, 'train_samples_per_second': 218.817, 'train_steps_per_second': 27.366, 'train_loss': 0.8890544841392708, 'epoch': 10.0}


No files have been modified since last commit. Skipping to prevent empty commit.


TrainOutput(global_step=17560, training_loss=0.8890544841392708, metrics={'train_runtime': 641.6774, 'train_samples_per_second': 218.817, 'train_steps_per_second': 27.366, 'total_flos': 3083788670469408.0, 'train_loss': 0.8890544841392708, 'epoch': 10.0})

In [34]:
# Replace this with your own checkpoint
model_checkpoint = "bert-finetuned-ner-lora"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Jino.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PER',
  'score': np.float32(0.9957055),
  'word': 'Jino',
  'start': 11,
  'end': 15}]